In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

%matplotlib inline
tf.config.set_visible_devices([], 'GPU')

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (150, 150))
            img_data = np.asarray(img)
            images.append(img_data)
    return images

In [3]:
def load_images_from_folder_2(folder):
    images = []
    for filename in os.listdir(folder):
        loaded_img = load_img(os.path.join(folder,filename), target_size=(150, 150))
        numpy_img = img_to_array(loaded_img)
        images.append(numpy_img)
    return images

In [4]:
control_images = load_images_from_folder_2('CT_NonCOVID')
covid_images = load_images_from_folder_2('CT_COVID')
all_images = control_images + covid_images
all_images = np.array(all_images)
# all_images = all_images.reshape((-1, 150, 150, 3))

In [5]:

all_images.shape

(746, 150, 150, 3)

In [6]:
# for i in range(4):
#     plt.subplot(2, 2, i + 1)
#     plt.imshow(np.uint8(all_images[i]))
# plt.show

In [7]:
control_labels = [0] * len(control_images)
covid_labels = [1] * len(covid_images)
label_series = pd.Series(control_labels + covid_labels)
print(len(all_images))
print(len(label_series))

746
746


In [8]:
X_train, X_test, y_train, y_test = train_test_split(all_images, label_series, test_size=0.25, random_state=42)

In [9]:
print(len(X_train))
print(len(y_train))

559
559


In [21]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


In [22]:
model.fit(X_train, y_train, batch_size=64, epochs=8, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
ba.update_state(y_test, model.predict(X_test))
print('COVID image detection accuracy:', ba.result().numpy())

Train on 559 samples
Epoch 1/8
559/559 [==============================] - 19s 33ms/sample - loss: 5.6254 - accuracy: 0.5009
Epoch 2/8
559/559 [==============================] - 18s 33ms/sample - loss: 1.2726 - accuracy: 0.5259
Epoch 3/8
559/559 [==============================] - 18s 32ms/sample - loss: 0.8688 - accuracy: 0.5259
Epoch 4/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.7043 - accuracy: 0.5725
Epoch 5/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.6645 - accuracy: 0.5921
Epoch 6/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.6223 - accuracy: 0.6404
Epoch 7/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.5647 - accuracy: 0.7174
Epoch 8/8
559/559 [==============================] - 18s 33ms/sample - loss: 0.5115 - accuracy: 0.7639
COVID image detection accuracy: 0.7379679


In [19]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

inp = tf.keras.layers.Input(shape=(150, 150, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inp, y)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, batch_size=64, epochs=10, use_multiprocessing=1)
ba = tf.keras.metrics.BinaryAccuracy()
ba.update_state(y_test, model.predict(X_test))
print('COVID image detection accuracy:', ba.result().numpy())

Train on 559 samples
Epoch 1/10
559/559 [==============================] - 5s 9ms/sample - loss: 30.7389 - accuracy: 0.5224
Epoch 2/10
559/559 [==============================] - 5s 9ms/sample - loss: 11.8919 - accuracy: 0.5259
Epoch 3/10
559/559 [==============================] - 5s 9ms/sample - loss: 4.8924 - accuracy: 0.5975
Epoch 4/10
559/559 [==============================] - 5s 9ms/sample - loss: 3.0190 - accuracy: 0.5957
Epoch 5/10
559/559 [==============================] - 5s 9ms/sample - loss: 1.5044 - accuracy: 0.6458
Epoch 6/10
559/559 [==============================] - 5s 9ms/sample - loss: 0.7812 - accuracy: 0.7084
Epoch 7/10
559/559 [==============================] - 5s 9ms/sample - loss: 0.4484 - accuracy: 0.8122
Epoch 8/10
559/559 [==============================] - 5s 9ms/sample - loss: 0.3386 - accuracy: 0.8444
Epoch 9/10
559/559 [==============================] - 5s 9ms/sample - loss: 0.2396 - accuracy: 0.8927
Epoch 10/10
559/559 [==============================] - 5s 9

In [14]:
model_url = 'https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1'
